In [120]:
import os
import torch
import monai
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd, Orientationd, ScaleIntensityRanged, CropForegroundd,
    RandCropByPosNegLabeld, RandFlipd, RandRotate90d, RandShiftIntensityd, EnsureTyped, DivisiblePadd
)
from monai.data import DataLoader, CacheDataset
from monai.networks.nets import SwinUNETR
from monai.utils import set_determinism
from monai.data.image_reader import NibabelReader
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import lightgbm as lgb

In [121]:
 # Define directories
train_path = 'dataset/MICCAI_BraTS2020_TrainingData/'
# val_path = 'dataset/MICCAI_BraTS2020_ValidationData/'
modality_keys = ["flair"]

In [122]:
def create_data_list(data_dir, patient_ids, modality_keys):
    data_list = []
    for patient in patient_ids:
        patient_dir = f"{data_dir}{patient}/"
        if os.path.isdir(patient_dir):
            data_dict = {key: os.path.join(patient_dir, f"{patient}_{key}.nii") for key in modality_keys}
            data_list.append(data_dict)

    return data_list

In [123]:
def preprocess_labels(csv_file_path):
    df = pd.read_csv(csv_file_path)
    df['Survival_days'] = pd.to_numeric(df['Survival_days'], errors='coerce')
    df = df.dropna(subset=['Survival_days'])
    df['Survival_days'] = df['Survival_days'].astype(int)
    df['Survival_class'] = df['Survival_days'].apply(lambda x: 0 if x < 300 else (1 if x < 450 else 2))
    return df['Survival_class'].values, df['Brats20ID'].values

In [124]:
all_labels, all_data_ids = preprocess_labels(f'{train_path}/survival_info.csv')
train_id, val_id, train_labels, val_labels = train_test_split(all_data_ids, all_labels, test_size=0.2, random_state=42)

train_data_list = create_data_list(train_path, train_id, modality_keys)
val_data_list = create_data_list(train_path, val_id, modality_keys)


# since we dont have lables for validation dataset not using it for validation
# val_data_list = create_data_list(val_path, modality_keys)
# val_labels = preprocess_labels(f'{val_path}/survival_evaluation.csv')



In [125]:
import nibabel as nib
import torch

# Load the NIfTI image
file_path = train_data_list[0]['flair']
img = nib.load(file_path)

# Get the image data as a numpy array
img_data = img.get_fdata()

# Convert the numpy array to a PyTorch tensor
img_tensor = torch.tensor(img_data, dtype=torch.float32)

# Print the shape of the tensor
print(f'Tensor shape: {img_tensor.shape}')


Tensor shape: torch.Size([240, 240, 155])


In [126]:
def get_transforms(modality_keys, pixdim=(2.0, 2.0, 2.0), spatial_size=(64, 64, 64)):
    transforms = Compose(
        [
            LoadImaged(keys=modality_keys, reader=NibabelReader()),
            EnsureChannelFirstd(keys=modality_keys),
            Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
            Orientationd(keys=modality_keys, axcodes="RAS"),
            ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
            DivisiblePadd(keys=modality_keys, k=32),
            RandCropByPosNegLabeld(keys=modality_keys, label_key=modality_keys[0], spatial_size=spatial_size, pos=1, neg=1, num_samples=1, image_key=modality_keys[0], image_threshold=0),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=0),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=1),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=2),
            RandRotate90d(keys=modality_keys, prob=0.5, max_k=3),
            RandShiftIntensityd(keys=modality_keys, offsets=0.10, prob=0.5),
            EnsureTyped(keys=modality_keys),
        ]
    )
    return transforms

In [127]:
def get_val_transforms(modality_keys, pixdim=(2.0, 2.0, 2.0)):
    transforms = Compose(
        [
            LoadImaged(keys=modality_keys, reader=NibabelReader()),
            EnsureChannelFirstd(keys=modality_keys),
            Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
            Orientationd(keys=modality_keys, axcodes="RAS"),
            ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
            DivisiblePadd(keys=modality_keys, k=32),
            EnsureTyped(keys=modality_keys),
        ]
    )
    return transforms

In [128]:
train_transforms = get_transforms(modality_keys)
val_transforms = get_val_transforms(modality_keys)


In [129]:
train_ds = CacheDataset(
    data=train_data_list,
    transform=train_transforms,
    cache_rate=0.5,
    num_workers=4,
)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4)


Loading dataset: 100%|██████████| 94/94 [00:54<00:00,  1.73it/s]


In [130]:
# for data in train_loader:
#     print(data['flair'].shape)

In [131]:
val_ds = CacheDataset(
    data=val_data_list,
    transform=val_transforms,
    cache_rate=0.5,
    num_workers=4,
)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=2)

Loading dataset: 100%|██████████| 23/23 [00:12<00:00,  1.83it/s]


In [132]:
# Load the pretrained model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SwinUNETR(
    img_size=(64, 64, 64),
    in_channels=in_channels,
    out_channels=out_channels,
    feature_size=48,
    use_checkpoint=True,
).to(device)


# Load the saved weights
modality_used = "_".join(modality_keys)
model_save_path = f"model_saved/swin_unetr_{modality_used}_temp.pth"

model.load_state_dict(torch.load(model_save_path))
model.eval()

/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


SwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(1, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=48, out_features=144, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=48, out_features=48, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=48, out_features=192, bias=True)
              (linear2): Linear(in_feature

In [133]:
# 2. Modify the SwinUNETR class to add a method for feature extraction
class FeatureExtractorSwinUNETR(SwinUNETR):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def extract_features(self, x):
        hidden_states = self.swinViT(x, self.normalize)
        enc0 = self.encoder1(x)
        enc1 = self.encoder2(hidden_states[0])
        enc2 = self.encoder3(hidden_states[1])
        enc3 = self.encoder4(hidden_states[2])
        dec3 = self.encoder10(hidden_states[3])
        return torch.cat([enc0, enc1, enc2, enc3, dec3], dim=1)

In [134]:
# Initialize the feature extractor with the same parameters as the SwinUNETR model
feature_extractor = FeatureExtractorSwinUNETR(
    img_size=(64, 64, 64),
    in_channels=len(modality_keys),
    out_channels=len(modality_keys),
    feature_size=48,
    use_checkpoint=True,
).to(device)

# Load the saved model state
feature_extractor.load_state_dict(torch.load(model_save_path))
feature_extractor.eval()

FeatureExtractorSwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(1, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=48, out_features=144, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=48, out_features=48, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=48, out_features=192, bias=True)
              (linear2): L

In [135]:
def extract_features_for_dataset(dataloader, feature_extractor):
    all_features = []

    feature_extractor.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting features"):
            inputs = torch.cat([batch[key] for key in modality_keys], dim=1).to(device)

            features = feature_extractor(inputs)

            # Flatten and concatenate features from all blocks
            # concat_features = torch.cat([f.view(f.size(0), -1) for f in features], dim=1)

            # all_features.append(concat_features.cpu().numpy())

    return np.vstack(all_features)


In [136]:

# Extract features for training and validation sets
train_features = extract_features_for_dataset(train_loader, feature_extractor)
val_features = extract_features_for_dataset(val_loader, feature_extractor)


Extracting features:   0%|          | 0/94 [00:00<?, ?it/s]/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/.local/lib/python3.8/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(




x_in torch.Size([2, 1, 64, 64, 64])
X layer1 torch.Size([2, 48, 32, 32, 32])

after downsampe1
 ==  torch.Size([2, 48, 32, 32, 32])
X layer2 torch.Size([2, 96, 16, 16, 16])


Extracting features:   0%|          | 0/94 [00:14<?, ?it/s]


RuntimeError: Given groups=1, weight of size [48, 1, 3, 3, 3], expected input[2, 96, 16, 16, 16] to have 1 channels, but got 96 channels instead

In [ ]:
# Create LightGBM datasets
train_data = lgb.Dataset(train_features, label=train_labels)
val_data = lgb.Dataset(val_features, label=val_labels)

In [ ]:
# Set parameters for LightGBM
params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [ ]:
# Train the model
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[val_data], early_stopping_rounds=10)


In [ ]:
# Make predictions
val_preds = bst.predict(val_features)
val_preds_classes = np.argmax(val_preds, axis=1)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(val_labels, val_preds_classes)
print(f"Validation Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(val_labels, val_preds_classes, target_names=le.classes_))